## Seminar 1: Fun with Word Embeddings (3 points)

Today we gonna play with word embeddings: train our own little embedding, load one from   gensim model zoo and use it to visualize text corpora.

This whole thing is gonna happen on top of embedding dataset.

__Requirements:__  `pip install --upgrade nltk gensim bokeh` , but only if you're running locally.

In [1]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

--2018-09-23 12:05:30--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Распознаётся www.dropbox.com (www.dropbox.com)… 162.125.66.1
Подключение к www.dropbox.com (www.dropbox.com)|162.125.66.1|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 301 Moved Permanently
Адрес: /s/dl/obaitrix9jyu84r/quora.txt [переход]
--2018-09-23 12:05:30--  https://www.dropbox.com/s/dl/obaitrix9jyu84r/quora.txt
Повторное использование соединения с www.dropbox.com:443.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://uc52ad093cffe97b15d450b23938.dl.dropboxusercontent.com/cd/0/get/ARNoOksp8eaYu-Ior2yA0-xpWCkTC7oacRNX7LG4rTLpBsMoFL67YRbBPkNmrRRW1Uv5FbFgDjHrqWlIPW8wsIaOOoilRWukWgjeLp2u4w9AgZrbgKG5rOlwFQzaASuFURl4Fo1SCZ7f1UFHCYAvieDibDb8J_zHaHm26jxglweYt81iQrNaHYSpRIt1HGew1jI/file?dl=1 [переход]
--2018-09-23 12:05:31--  https://uc52ad093cffe97b15d450b23938.dl.dropboxusercontent.com/cd/0/get/ARNoOksp8eaYu-Ior2yA0-xpWCkTC7oacRNX7LG4rTLpBsMoFL67YRbBPkNmrRRW1Uv5FbFgD

In [2]:
import numpy as np

data = list(open("./quora.txt"))
data[50]

"What TV shows or books help you read people's body language?\n"

__Tokenization:__ a typical first step for an nlp task is to split raw data into words.
The text we're working with is in raw format: with all the punctuation and smiles attached to some words, so a simple str.split won't do.

Let's use __`nltk`__ - a library that handles many nlp tasks like tokenization, stemming or part-of-speech tagging.

In [3]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [4]:
# TASK: lowercase everything and extract tokens with tokenizer. 
# data_tok should be a list of lists of tokens for each line in data.

data_tok = [tokenizer.tokenize(string.lower()) for string in data]
print(data_tok[50])

['what', 'tv', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [5]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

In [6]:
print([' '.join(row) for row in data_tok[:2]])

["can i get back with my ex even though she is pregnant with another guy ' s baby ?", 'what are some ways to overcome a fast food addiction ?']


__Word vectors:__ as the saying goes, there's more than one way to train word embeddings. There's Word2Vec and GloVe with different objective functions. Then there's fasttext that uses character-level models to train word embeddings. 

The choice is huge, so let's start someplace small: __gensim__ is another nlp library that features many vector-based models incuding word2vec.

In [7]:
len(data_tok)

537272

In [19]:
from gensim.models import Word2Vec
model = Word2Vec(data_tok, 
                 size=32,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv  # define context as a 5-word window around the target word

In [20]:
# now you can get word vectors !
model.get_vector('anything')

array([ 1.2068808 ,  0.5594116 , -2.3684304 , -0.805223  , -0.44294024,
       -0.9624045 ,  3.5702102 ,  1.6651751 , -1.3845338 ,  0.28192145,
        2.5823526 ,  0.14318708,  0.6058058 , -2.2541125 , -1.6897236 ,
        4.8350787 , -4.49005   , -0.6664193 ,  1.306786  ,  1.2508442 ,
       -2.0612147 , -0.95687133,  0.32010427,  1.0718659 , -0.16227345,
        0.75969017,  3.030435  ,  1.3498498 , -0.01887888, -2.568705  ,
       -2.9857345 ,  1.404426  ], dtype=float32)

In [21]:
# or query similar words directly. Go play with it!
model.most_similar('cat')

/Users/romanmarakulin/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('bitsat', 0.8125330209732056),
 ('gmat', 0.7982620596885681),
 ('gre', 0.7935650944709778),
 ('usmle', 0.7860528230667114),
 ('ielts', 0.7828070521354675),
 ('sat', 0.7823654413223267),
 ('xat', 0.7798899412155151),
 ('percentile', 0.7787757515907288),
 ('toefl', 0.7715185880661011),
 ('aipmt', 0.7609981298446655)]

### Using pre-trained model

Took it a while, huh? Now imagine training life-sized (100~300D) word embeddings on gigabytes of text: wikipedia articles or twitter posts. 

Thankfully, nowadays you can get a pre-trained word embedding model in 2 lines of code (no sms required, promise).

In [22]:
import gensim.downloader as api
model = api.load('glove-twitter-100')

In [23]:
model.most_similar('cat')

/Users/romanmarakulin/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('dog', 0.875208854675293),
 ('kitty', 0.8015091419219971),
 ('pet', 0.7986468076705933),
 ('cats', 0.7979425191879272),
 ('kitten', 0.7936833500862122),
 ('puppy', 0.7702749371528625),
 ('monkey', 0.7584263682365417),
 ('bear', 0.7507943511009216),
 ('dogs', 0.7460063695907593),
 ('pig', 0.7117345929145813)]

In [24]:
model.most_similar(positive=["coder", "money"], negative=["brain"])

/Users/romanmarakulin/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('broker', 0.5820155739784241),
 ('bonuses', 0.5424473285675049),
 ('banker', 0.5385112762451172),
 ('designer', 0.5197198987007141),
 ('merchandising', 0.4964233338832855),
 ('treet', 0.49220195412635803),
 ('shopper', 0.4920561611652374),
 ('part-time', 0.4912828207015991),
 ('freelance', 0.4843311905860901),
 ('aupair', 0.4796452522277832)]

### Visualizing word vectors

One way to see if our vectors are any good is to plot them. Thing is, those vectors are in 30D+ space and we humans are more used to 2-3D.

Luckily, we machine learners know about __dimensionality reduction__ methods.

Let's use that to plot 1000 most frequent words

In [25]:
words = sorted(model.vocab.keys(), 
               key=lambda word: model.vocab[word].count,
               reverse=True)[:1000]

print(words[::100])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


In [28]:
# for each word, compute it's vector with model
word_vectors = np.array([model.get_vector(word) for word in words])

In [29]:
assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 100)
assert np.isfinite(word_vectors).all()

#### Linear projection: PCA

The simplest linear dimensionality reduction method is __P__rincipial __C__omponent __A__nalysis.

In geometric terms, PCA tries to find axes along which most of the variance occurs. The "natural" axes, if you wish.

<img src="https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/pca_fish.png" style="width:30%">


Under the hood, it attempts to decompose object-feature matrix $X$ into two smaller matrices: $W$ and $\hat W$ minimizing _mean squared error_:

$$\|(X W) \hat{W} - X\|^2_2 \to_{W, \hat{W}} \min$$
- $X \in \mathbb{R}^{n \times m}$ - object matrix (**centered**);
- $W \in \mathbb{R}^{m \times d}$ - matrix of direct transformation;
- $\hat{W} \in \mathbb{R}^{d \times m}$ - matrix of reverse transformation;
- $n$ samples, $m$ original dimensions and $d$ target dimensions;



In [30]:
from sklearn.decomposition import PCA

# map word vectors onto 2d plane with PCA. Use good old sklearn api (fit, transform)
# after that, normalize vectors to make sure they have zero mean and unit variance
word_vectors_pca = PCA(n_components=2).fit_transform(word_vectors)

# and maybe MORE OF YOUR CODE here :)

In [31]:
assert word_vectors_pca.shape == (len(word_vectors), 2), "there must be a 2d vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1 - word_vectors_pca.std(0) < 1e-5)), "points must have unit variance"

#### Let's draw it!

In [32]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [33]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

# hover a mouse over there and see if you can identify the clusters

Figure(id='82a0c194-443d-4d59-9649-5d880f10efb9', ...)

### Visualizing neighbors with t-SNE
PCA is nice but it's strictly linear and thus only able to capture coarse high-level structure of the data.

If we instead want to focus on keeping neighboring points near, we could use TSNE, which is itself an embedding method. Here you can read __[more on TSNE](https://distill.pub/2016/misread-tsne/)__.

In [34]:
from sklearn.manifold import TSNE

# map word vectors onto 2d plane with TSNE. hint: use verbose=100 to see what it's doing.
# normalize them as just lke with pca


word_tsne = TSNE(verbose=True).fit_transform(word_vectors)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1000 samples in 0.004s...
[t-SNE] Computed neighbors for 1000 samples in 0.176s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1000
[t-SNE] Mean sigma: 1.716134
[t-SNE] KL divergence after 250 iterations with early exaggeration: 69.605682
[t-SNE] Error after 1000 iterations: 1.034775


In [35]:
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color='green', token=words)

Figure(id='1c84a0a5-e84a-4476-af5b-7fb1432eefe1', ...)

### Visualizing phrases

Word embeddings can also be used to represent short phrases. The simplest way is to take __an average__ of vectors for all tokens in the phrase with some weights.

This trick is useful to identify what data are you working with: find if there are any outliers, clusters or other artefacts.

Let's try this new hammer on our data!


In [38]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros
    
    vector = np.zeros([model.vector_size], dtype='float32')
    vector_count = 0
    for token in tokenizer.tokenize(phrase.lower()):
        if token in model.vocab:
            vector += model.get_vector(token)
            vector_count += 1
    if vector_count:
        vector = vector / vector_count
    return vector

In [39]:
vector = get_phrase_embedding("I'm very sure. This never happened to me before...")

assert np.allclose(vector[::10],
                   np.array([ 0.31807372, -0.02558171,  0.0933293 , -0.1002182 , -1.0278689 ,
                             -0.16621883,  0.05083408,  0.17989802,  1.3701859 ,  0.08655966],
                              dtype=np.float32))

In [42]:
# let's only consider ~5k phrases for a first run.
chosen_phrases = data[::len(data) // 1000]

# compute vectors for chosen phrases
phrase_vectors = np.array([get_phrase_embedding(phrase) for phrase in chosen_phrases])

In [43]:
assert isinstance(phrase_vectors, np.ndarray) and np.isfinite(phrase_vectors).all()
assert phrase_vectors.shape == (len(chosen_phrases), model.vector_size)

In [44]:
# map vectors into 2d space with pca, tsne or your other method of choice
# don't forget to normalize

phrase_vectors_2d = TSNE(verbose=1000).fit_transform(phrase_vectors)

phrase_vectors_2d = (phrase_vectors_2d - phrase_vectors_2d.mean(axis=0)) / phrase_vectors_2d.std(axis=0)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1001 samples in 0.002s...
[t-SNE] Computed neighbors for 1001 samples in 0.169s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1001
[t-SNE] Computed conditional probabilities for sample 1001 / 1001
[t-SNE] Mean sigma: 0.477874
[t-SNE] Computed conditional probabilities in 0.032s
[t-SNE] Iteration 50: error = 79.8647537, gradient norm = 0.3124800 (50 iterations in 4.666s)
[t-SNE] Iteration 100: error = 80.4321899, gradient norm = 0.3159084 (50 iterations in 4.862s)
[t-SNE] Iteration 150: error = 79.8531723, gradient norm = 0.3076565 (50 iterations in 4.954s)
[t-SNE] Iteration 200: error = 81.3570328, gradient norm = 0.3026519 (50 iterations in 4.931s)
[t-SNE] Iteration 250: error = 81.3081207, gradient norm = 0.2982652 (50 iterations in 4.841s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 81.308121
[t-SNE] Iteration 300: error = 2.2080016, gradient norm = 0.0037695 (50 iterations in 3.747s)

In [45]:
draw_vectors(phrase_vectors_2d[:, 0], phrase_vectors_2d[:, 1],
             phrase=[phrase[:50] for phrase in chosen_phrases],
             radius=20,)

Figure(id='0f3fd755-855b-4821-86dc-690768efa19d', ...)

Finally, let's build a simple "similar question" engine with phrase embeddings we've built.

In [47]:
# compute vector embedding for all lines in data
data_vectors = np.array([get_phrase_embedding(l) for l in data])

In [96]:
def cosine_distance(X, y):
    epsilon = 10 ** -40 # to avoid divide by 0
    return 1. - X.dot(y) / np.maximum(np.linalg.norm(X, axis=1), epsilon) / np.maximum(np.linalg.norm(y), epsilon)

# check:
cosine_distance(data_vectors[:3], data_vectors[5])

array([0.07947016, 0.05718321, 0.12193191], dtype=float32)

In [97]:
def find_nearest(query, k=10):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    query_vector = get_phrase_embedding(query)
    distances = cosine_distance(data_vectors, query_vector)
    
    # find wanted idxs
    found_idxs = np.argpartition(distances, k)[:k]
    # should sort distances of found_idxs
    right_order = np.argsort(distances[found_idxs])
    found_idxs = [found_idxs[idx] for idx in right_order]
    return [data[idx] for idx in found_idxs]

In [98]:
results = find_nearest(query="How do i enter the matrix?", k=10)

print(''.join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[0] == 'How do I get to the dark web?\n'
assert results[3] == 'What can I do to save the world?\n'

How do I get to the dark web?
What should I do to enter hollywood?
How do I use the Greenify app?
What can I do to save the world?
How do I win this?
How do I think out of the box? How do I learn to think out of the box?
How do I find the 5th dimension?
How do I use the pad in MMA?
How do I estimate the competition?
What do I do to enter the line of event management?



In [99]:
find_nearest(query="How does Trump?", k=10)

['What does Donald Trump think about Israel?\n',
 'What books does Donald Trump like?\n',
 'What does Donald Trump think of India?\n',
 'What does India think of Donald Trump?\n',
 'What does Donald Trump think of China?\n',
 'What does Donald Trump think about Pakistan?\n',
 'What companies does Donald Trump own?\n',
 'What does Dushka Zapata think about Donald Trump?\n',
 'How does it feel to date Ivanka Trump?\n',
 'What does salesforce mean?\n']

In [100]:
find_nearest(query="Why don't i ask a question myself?", k=10)

["Why don't I get a date?\n",
 "Why do you always answer a question with a question? I don't, or do I?\n",
 "Why can't I ask a question anonymously?\n",
 "Why don't I get a girlfriend?\n",
 "Why don't I have a boyfriend?\n",
 "I don't have no question?\n",
 "Why can't I take a joke?\n",
 "Why don't I ever get a girl?\n",
 "Can I ask a girl out that I don't know?\n",
 "Why don't I have a girlfriend?\n"]

__Now what?__
* Try running TSNE on all data, not just 1000 phrases
* See what other embeddings are there in the model zoo: `gensim.downloader.info()`
* Take a look at [FastText](https://github.com/facebookresearch/fastText) embeddings
* Optimize find_nearest with locality-sensitive hashing: use [nearpy](https://github.com/pixelogik/NearPy) or `sklearn.neighbors`.